Si $f_\theta : \mathbb{R}_{\geq 0} \times \Omega \rightarrow \mathbb{R}$ es la red neuronal: $f_\theta( t, \vec{x})$

Una opcion es minimizar el BE? Pero aca esta fijo el $t$

$$ \underset{u: \Omega \rightarrow \mathbb{R}}{\text{min}} \quad \quad \int_\Omega \left | t(id - \nabla \cdot \circ \, P_{\nabla \psi} \,\nabla) f_\theta - u_0 \right | \quad d\vec{x}.$$

La otra es la mejor... que es minimizar la ecuacion del calor de verdad, pero intrinseco a $\partial \Omega$:

$$\begin{matrix} & \frac{\partial u}{\partial t} &= \Delta_{\partial \Omega} u \\ \text{sujeto a:} & u(\gamma) &= \delta(\gamma) \end{matrix}$$

Con $\delta(\cdot)$ la funcion delta de Dirac.

In [ ]:
import numpy as np
import torch
from src.model import SIREN
import random
import src.diff_operators as dif

In [ ]:
SDF_model = SIREN(
        n_in_features= 3,
        n_out_features=1,
        hidden_layer_config=[512]*4,
        w0=30,
        ww=None,
        activation= 'sine'
)
SDF_model.load_state_dict( torch.load('results/torus/experiment/models/model_best.pth', weights_only=True))

device = torch.device(0)
SDF_model.to(device)

In [ ]:
seed = 123 
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


heat_kernel = SIREN(
    n_in_features= 4, # tenemos 4 porque depende del tiempo tambien
    n_out_features=1,
    hidden_layer_config=[16] * 4,
    w0=30,
    ww=None,
    activation='sine'
)

In [ ]:
heat_kernel.to(device)

losses = dict()
best_loss = np.inf
best_weights = None
current_lr = 1e-4
epochs = 1000

optimizer = torch.optim.Adam(
            lr=current_lr,
            params=heat_kernel.parameters()
        )

for g in optimizer.param_groups:
    g['lr'] = current_lr


In [ ]:
def sample_data():
    boundary_data = 

In [ ]:
for epoch in range(epochs):
    # zero the parameter gradients
    optim.zero_grad()
    
    boundary_data, domain_data = 
    # forward + backward + optimize
    input_data = input_data.to( device )
    normals = normals.to(device)
    sdfs = sdfs.to(device)
    
    loss = loss_fn( 
        model, 
        input_data, 
        {'normals': normals, 'sdfs': sdfs}, 
        loss_weights
    )

    wandb.log( loss )

    train_loss = torch.zeros((1, 1), device=device)
    for it, l in loss.items():
        train_loss += l
        # accumulating statistics per loss term
        if it not in running_loss:
            running_loss[it] = l.item()
        else:
            running_loss[it] += l.item()

    train_loss.backward()
    optim.step()

    # accumulate statistics
    for it, l in running_loss.items():
        if it in losses:
            losses[it][epoch] = l
        else:
            losses[it] = [0.] * epochs
            losses[it][epoch] = l

    epoch_loss = 0
    for k, v in running_loss.items():
        epoch_loss += v
    epoch_loss /=+ dataset.batchesPerEpoch
    print(f"Epoch: {epoch} - Loss: {epoch_loss} - Learning Rate: {current_lr:.3e}")

In [ ]:
import numpy as np
import torch
from src.model import SIREN
from src.obj import load
import meshplot as mp
import src.diff_operators as dif

In [ ]:
model = SIREN(
        n_in_features= 3,
        n_out_features=1,
        hidden_layer_config=[512]*4,
        w0=30,
        ww=None,
        activation= 'sine'
)
model.load_state_dict( torch.load('results/bunny/experiment/models/model_best.pth', weights_only=True))

device_torch = torch.device(0)
model.to(device_torch)

In [ ]:
def grad_desc( model, samples, max_batch=64**2, device=torch.device(0), iterations = 3 ):
    # samples = ( amount_samples, 3 )
    amount_samples = samples.shape[0]

    head = 0
    X = samples.copy()

    for i in range( iterations):
        print(f'Iteration: {i}')
        mean_distance = 0
        while head < amount_samples:
            
            inputs_subset = torch.from_numpy( X[head:min(head + max_batch, amount_samples), :] ).to(device).unsqueeze(0).float()

            x, y = model(inputs_subset).values()

            mean_distance += torch.sum(y).detach().cpu().numpy()

            grad_psi = torch.nn.functional.normalize( dif.gradient(y,x), dim=-1 )

            X[head:min(head + max_batch, amount_samples)] -= (y *  grad_psi).squeeze(0).detach().cpu().numpy()

            head += max_batch

        print(f'Mean distance: { mean_distance / amount_samples }')

    return X

pc, _ = load( 'data/bunny/bunny_pc.obj' )
#pc = pc[np.random.choice( np.arange(100000), 10000),: ]
pc = grad_desc( model, pc, iterations=1 )

In [ ]:
t = 1
EPSILON = 0.001

u0 = lambda x, p: torch.exp( - torch.sum( (x - p)**2, dim=-1) / (EPSILON ** 2) )[...,None] / ( EPSILON )
grad_u0 = lambda x, p: (-2 / (EPSILON ** 2)) * u0(x, p) * (x - p)

In [ ]:
def project( projectees, projectors ):
    # asumiendo ||projectors|| = 1
    return projectees -  torch.sum(projectors * projectees, dim=-1)[...,None] * projectors

def computeX( model, samples, p, u0, grad_u0, max_batch=64**2, device=torch.device(0) ):
    # samples = ( amount_samples, 3 )    
    head = 0
    amount_samples = samples.shape[0]

    uts = np.zeros( (amount_samples, 1))
    grads_psi = np.zeros( (amount_samples, 3))
    laplacians_psi = np.zeros( (amount_samples, 1))
    X = np.zeros( (amount_samples, 3))
    divX = np.zeros( (amount_samples, 1))

    ps = torch.from_numpy( np.tile(p, (max_batch, 1)) ).to(device_torch)
    i = 0
    while head < amount_samples:
        print(f'Iteration: {i}')
        
        inputs_subset = torch.from_numpy( samples[head:min(head + max_batch, amount_samples), :] ).to(device).unsqueeze(0).float()

        x, y =  model(inputs_subset).values()

        grad_psi = dif.gradient(y,x)

        ps_ss = ps[:inputs_subset.shape[1],:].unsqueeze(0)

        # Forward Euler
        proj_grad_u0 = project( grad_u0( inputs_subset, ps_ss ), grad_psi )
        F_u0 = dif.divergence( proj_grad_u0 , x )
        u0s = u0( inputs_subset, ps_ss )
        ut = u0s + t * F_u0

        # RK4: ( out of memory... lo mata hacer el gradiente de u para calcular X.. es un monton)
        #k1 = dif.divergence( project( grad_u0( inputs_subset, ps_ss ), grad_psi ) , x )
        #k2 = dif.divergence( project( dif.gradient(u0s + 0.5 * t * k1 , x), grad_psi ), x )
        #k3 = dif.divergence( project( dif.gradient(u0s + 0.5 * t * k2 , x), grad_psi ), x )
        #k4 = dif.divergence( project( dif.gradient(u0s + t * k3 , x), grad_psi ), x )
        #ut = u0s + (t/6) * (k1 + 2*k2 + 2*k3 + k4)

        # Heuns:
        #k1 = dif.divergence( proj_grad_u0 , x )
        #k2 = dif.divergence( project( dif.gradient(u0s + t * k1 , x), grad_psi ), x )
        #ut = u0s + (t/2) * (k1 + k2)
        
        uts[head:min(head + max_batch, amount_samples)] = ut.detach().cpu().squeeze(0).numpy()

        #X_ss = -1 * torch.nn.functional.normalize( proj_grad_u0 + project( dif.gradient( (t/2) * (k1 + k2), x ), grad_psi ), dim=-1, eps=1e-30 )
        X_ss = -1 * torch.nn.functional.normalize( proj_grad_u0 + t * project( dif.gradient( F_u0 , x), grad_psi ), dim=-1 , eps=1e-30 )
        #X_ss =  torch.nn.functional.normalize( proj_grad_u0 , dim=-1 , eps=1e-30 )
        
        X[head:min(head + max_batch, amount_samples)] = X_ss.detach().cpu().squeeze(0).numpy()
        divX[head:min(head + max_batch, amount_samples)] = dif.divergence( X_ss, x ).detach().cpu().squeeze(0).numpy()

        laplacians_psi[head:min(head + max_batch, amount_samples)] = dif.divergence( grad_psi, x ).detach().cpu().squeeze(0).numpy()
        grads_psi[head:min(head + max_batch, amount_samples)] = grad_psi.detach().cpu().squeeze(0).numpy()

        head += max_batch
        i += 1

        torch.cuda.empty_cache()

    return grads_psi, laplacians_psi, uts, X, divX

grads_psi, laplacians_psi, ut, X, divX = computeX( model, pc, pc[47,:], u0, grad_u0, max_batch=64**2 )